In [ ]:
import tensorflow as tf
import os
import pathlib
import IPython
import sys
from sklearn.model_selection import train_test_split
import cv2
import numpy as np

In [ ]:
# from google.colab import drive             
# drive.mount('/content/drive')

In [ ]:
data_dir = pathlib.Path('/home/samer/Desktop/Tobacco3482-jpg/')

img_data_array = []
class_name = []

for dir1 in os.listdir(data_dir):
    for file in os.listdir(os.path.join(data_dir, dir1)):
        image_path = os.path.join(data_dir, dir1, file)
        image = cv2.imread(image_path, cv2.COLOR_BGR2RGB)
        if image is None:
            continue
        image = cv2.resize(image, (1200, 1200))
        image = np.array(image)
        image = image.astype("float32")
        image /= 255
        if len(image.shape) < 3:
            continue
        else:
            img_data_array.append(image)
            class_name.append(dir1)


# Split data into training and testing sets
labels = tf.keras.utils.to_categorical(class_name)
x_train, x_test, y_train, y_test = train_test_split(np.array(image_data_array), np.array(class_name), test_size=0.3)


# data_path_test = pathlib.Path('/content/drive/MyDrive/archiveX3/Testing/')
# AUTOTUNE=tf.data.AUTOTUNE

# dataset_path = tf.keras.utils.image_dataset_from_directory(        # Training dataset
#     data_path,
#     labels= 'inferred',
#     validation_split=0.2,
#     subset='training',
#     seed= 1,
#     batch_size=5,
#     image_size=(299, 299),
#     color_mode="rgb",
#     shuffle=True)#.cache().prefetch(buffer_size=AUTOTUNE)

# dataset_path_val = tf.keras.utils.image_dataset_from_directory(      #Validation dataset
#     data_path,
#     labels= 'inferred',
#     validation_split=0.2,
#     subset='validation',
#     seed= 2,
#     batch_size=5,
#     image_size=(299, 299),
#     color_mode="rgb",
#     shuffle=True)#.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
base_model = tf.keras.applications.inception_v3.InceptionV3(
    input_shape = (299, 299, 3),
    include_top = False, 
    weights = "imagenet"                  
)

In [ ]:
saving_path = pathlib.Path('/home/samer/Desktop/Tobacco3482-jpg/SavedBaseModel.h5')

base_model.save(saving_path)

In [ ]:
model_path = pathlib.Path('/home/samer/Desktop/Tobacco3482-jpg/SavedBaseModel.h5')

base_model = tf.keras.models.load_model(model_path)                 #Loading base_model

base_model.trainable = False                                        # Setting the model as non-trainable

rescaling = tf.keras.Sequential([                                        
  tf.keras.layers.Rescaling(scale=1 / 127.5, offset=-1)             # Rescaling to (1, -1) range required for inceptionV3 model
])
augmentation = tf.keras.Sequential([                                # Applying augmentations o the images
   tf.keras.layers.RandomFlip("horizontal"),
   tf.keras.layers.RandomRotation(0.1)
])


inputs = tf.keras.Input(shape=(1200, 1200, 3))
x = augmentation(inputs)
x = rescaling(x)
x = base_model(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
outputs = tf.keras.layers.Dense(4, activation='softmax')(x)                   # "softmax" in the final layer for decision making
model = tf.keras.Model(inputs, outputs)


model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0005),        # compiling with low learning rate
    loss=tf.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy'],
    run_eagerly = True)   

model.fit(                                                               # fitting the whole model for non-trainable base
    dataset_path,
    epochs=1,)   

